Created a notebook so it can be organized. Started Aug 12, 2022

# Set up

In [2]:
#@title ## Base imports
import os
import sys
import numpy as np
import scipy
import sklearn
import sklearn.linear_model
import pandas as pd
import plotly.express as px

import warnings
import requests
import urllib.request
import json

In [3]:
#@title ## Option 1) Mount google drive and import my code

mountpoint_folder_name = "gdrive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/GeospatialAnalysis" #@param {type:"string"}
project_path_full = os.path.join("/content/",mountpoint_folder_name,
                        "MyDrive",project_path_within_drive)
try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name

    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

/root
Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1okL5s1HTQUWqKodPSVNP_RsK0o81IH5E/PythonProjects/GeospatialAnalysis


In [ ]:
#@title ## Option 2) Clone project files from GitHub

!git clone https://github.com/ryerrabelli/GeospatialAnalysis.git

project_path_full = os.path.join("/content/","GeospatialAnalysis/")
sys.path.insert(1,project_path_full)
%cd GeospatialAnalysis
print(sys.path)

Cloning into 'GeospatialAnalysis'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 198 (delta 0), reused 3 (delta 0), pack-reused 192
Receiving objects: 100% (198/198), 31.41 MiB | 8.15 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/GeospatialAnalysis
['', '/content/GeospatialAnalysis', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


# Helper functions

In [4]:
years = np.arange(2015,2019+1)

#@markdown # calc_regression()
def calc_regression(y, x):
    import collections
    regress = scipy.stats.linregress(x, y=y)
    # R = Pearson coefficient
    # p indicates p-value
    # Use ordered dict to retain order
    return collections.OrderedDict({
        "Mean": np.mean(y, axis=0),
        "Sum": np.sum(y, axis=0),
        "Slope": regress.slope, 
        "Intercept": regress.intercept, 
        "R": regress.rvalue, 
        "p": regress.pvalue, 
        })


"""
requests.get('http://172.28.0.2:9000/api/sessions').json() =
[{'id': 'e0a49454-e812-4d99-aa6f-9d7b80a9616c',
  'kernel': {'connections': 1,
   'execution_state': 'busy',
   'id': '6fd9c8b4-6573-4ffa-a7d6-f56803a0092c',
   'last_activity': '2021-12-01T08:12:08.239708Z',
   'name': 'python3'},
  'name': 'ModelInversions.ipynb',
  'notebook': {'name': 'ModelInversions.ipynb',
   'path': 'fileId=1ZAqQEIxR08eODSPEHvKPwbZoioVdV8L9'},
  'path': 'fileId=1ZAqQEIxR08eODSPEHvKPwbZoioVdV8L9',
  'type': 'notebook'}]
"""
#notebook_filename = requests.get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
notebook_filename="2022_12_29-RSY-geospatial_ENT_analysis_v04.ipynb"
#@markdown # get_path_to_save()
def get_path_to_save(file_prefix="", save_filename:str=None, save_in_subfolder:str=None, extension="png", create_folder_if_necessary=True):
    save_path = ["outputs",
                f"{notebook_filename.split('.',1)[0]}",  # use split to remove file extension
                ]
    if save_in_subfolder is not None:
        if isinstance(save_in_subfolder, (list, tuple, set, np.ndarray) ):
            save_path.append(**save_in_subfolder)
        else:  # should be a string then
            save_path.append(save_in_subfolder)
    save_path = os.path.join(*save_path)
    if not os.path.exists(save_path) and create_folder_if_necessary:
        os.makedirs(save_path)
    return os.path.join(save_path, file_prefix+save_filename+"."+extension)

#@markdown # save_df()
def save_df(df, file_name:str, ):
    df.to_excel( get_path_to_save(save_filename=file_name, extension="xlsx") )
    df.to_csv( get_path_to_save(save_filename=file_name, extension="csv") )

#@markdown # save_figure()
def save_figure(fig, file_name:str, animated=False):
    """
    fig is of type plotly.graph_objs._figure.Figure,
    Requires kaleido to be installed
    """
    fig.write_html( get_path_to_save(save_filename=file_name, extension="html") )
    if not animated:
        fig.write_image( get_path_to_save(save_filename=file_name, extension="svg"))
        fig.write_image( get_path_to_save(save_filename=file_name, extension="png") )
        fig.write_image( get_path_to_save(save_filename=file_name, extension="jpg") )


# Procedures analysis

## Load ENT procedures df from csv file
This is specifically a wide type df so it is one row per procedure with years as different columns.To understand what is meant by long type and wide type dataframes, see https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57

The slope given in the csv file is actually the inverse slope. We need to either recalculate it or invert it. I will just recalculate all the regression values.

In [27]:
df_procedures_orig = pd.read_csv("data/1_renamed/procedure_specific_data.csv", 
                           dtype={
                               "Specialty": str,
                               "Category": str,
                               "HCPCS Code": str,
                               "Total Number of Services": np.int64,
                               **{f"Total Number of Services: {year}": np.int64 for year in range(2015,2019+1)}
                               })  # gets per healthcare code info

## Clean df and recalculate regression

In [33]:
df_procedures_clean = df_procedures_orig.set_index(["Specialty","Group","HCPCS Code", "HCPCS Description"])

# Remove the "amount" word 
df_procedures_clean.columns = [col.replace("Total Medicare Payment Amount","Total Medicare Payment") for col in df_procedures_clean.columns]
# Drop columns besides the individual year ones. Will recalculate the other ones as a quality assurance check.
df_procedures_clean = df_procedures_clean.drop(columns=[col for col in df_procedures_clean.columns if ("slope" in col.lower() or "pearson" in col.lower() or ":" not in col)] )

# Rename the columns so they can be split  easier. The 20 is the first two digits of the year columns
df_procedures_clean.columns = [col.replace(": ",": : ").replace(": 20","Annual: 20") for col in df_procedures_clean.columns]

# Make Multiindex
df_procedures_clean.columns = pd.MultiIndex.from_tuples([tuple(col.split(": ")) if ":" in col else (col,"","") for col in df_procedures_clean.columns], names=["Category","Stat","Year"])
df_procedures_clean = df_procedures_clean[sorted(df_procedures_clean)]  # rearrange cols alphabetically


categories = df_procedures_clean.columns.levels[0]  #["Total Number of Services", "Total Medicare Payment Amount"]

# Calculate regression and sum and mean from individual year later
df_procedures_recalc = df_procedures_clean.copy()
for category in categories:
    new_df = df_procedures_recalc[(category,"Annual")].apply(calc_regression,axis=1, result_type="expand", args=(years,) )
    df_procedures_recalc[[(category,"Overall",new_col) for new_col in new_df.columns ]]=new_df
    #df_procedures_recalc[(category,"","Slope")]=df_procedures_recalc[(category,"Annual")].apply(calc_regression,axis=1)

# rearrange cols alphabetically, but only by the first two elements of the each column's name tuple
# This allows the order of the newly added columns to remain relative to themselves, but be rearranged relative to the other columns
df_procedures_recalc = df_procedures_recalc[sorted(df_procedures_recalc.columns, key=(lambda x: x[0:2]))]  

#df_procedures = df_procedures.sort_values(by=("Total Number of Services","","Sum"), ascending=False)  # sort rows by volume 
df_procedures_recalc = df_procedures_recalc.sort_values(by=("Total Medicare Payment","Overall","Mean"), ascending=False)  # sort rows by volume 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


In [35]:
df_procedures_recalc

Category                                                                                      Total Medicare Payment  \
Stat                                                                                                          Annual   
Year                                                                                                            2015   
Specialty                 Group HCPCS Code HCPCS Description                                                           
Facial plastics           A     14060      Tissue transfer repair of wound (10 sq centimet...             7253164.21   
Otology                   NaN   69930      Implantation of cochlear device                                4164557.97   
Rhinology                 J     31267      Removal of nasal sinus tissue using an endoscope               2454520.71   
Facial plastics/Rhinology NaN   30520      Reshaping of nasal cartilage                                   3822391.75   
Rhinology                 H     30140      Removal of nasal air passage                                   4090571.97   
Facial plastics           NaN   15260      Relocation of patient skin to nose, ears, eyeli...             3816163.30   
                          A     14301      Tissue transfer repair of wound (30.1 to 60.0 s...             3422400.08   
                                14040      Tissue transfer repair of wound (10 sq centimet...             4175118.98   
Laryngology               E     31575      Diagnostic examination of voice box using flexi...             4095655.56   
Rhinology                 NaN   31276      Exploration of nasal sinus using an endoscope                  3551553.36   
Facial plastics           A     14041      Tissue transfer repair of wound (10.1 to 30.0 s...             2481173.43   
Head & neck               NaN   60500      Removal or exploration of parathyroid glands                   1466292.92   
Rhinology                 NaN   31231      Diagnostic examination of nasal passages using ...             1561172.64   
Laryngology               E     31579      Examination to assess movement of vocal cord fl...             1329457.32   
Otology                   F     69436      Incision of eardrum with insertion of eardrum t...             1291093.18   
Rhinology                 NaN   31256      Incision of nasal sinus using an endoscope                     1021363.93   
Laryngology               E     31525      Diagnostic examination of voice box using an en...              548628.32   
Facial plastics           NaN   11042      Removal of skin and tissue first 20 sq cm or less               509299.24   
Rhinology                 NaN   31237      Biopsy or removal of nasal polyp or tissue usin...              689656.89   
                                30465      Widening of nasal passage                                       423255.25   
Facial plastics           NaN   15004      Preparation of graft site of face, scalp, eyeli...              492031.70   
                                20926      Tissue graft                                                    426939.03   
Otology                   NaN   21235      Obtaining ear cartilage for grafting                            268245.90   
Rhinology                 NaN   31254      Partial removal of nasal sinus using an endoscope               278635.35   
                          J     31288      Removal of nasal sinus tissue using an endoscope                460441.25   
Head & neck               K     60240      Removal of thyroid                                              559710.52   
Laryngology               NaN   31571      Injection of vocal cords using an endoscope wit...              337017.46   
Head & neck               NaN   31541      Removal of growth of tongue and/or vocal cord s...              428916.69   
Facial plastics           NaN   15120      Skin graft of face, scalp, eyelids, mouth, neck...              249466.87   
Rhinology                 C     31253   

In [ ]:
with pd.option_context('display.float_format', '{:,.2f}'.format):
    display(df_procedures_recalc)

save_df(df_procedures_recalc, "df_procedures_recalc")

Category                                                      Total Medicare Payment  \
Stat                                                                          Annual   
Year                                                                            2015   
HCPCS Code HCPCS Description                                                           
14060      Tissue transfer repair of wound (10 sq centimet...           7,253,164.21   
69930      Implantation of cochlear device                              4,164,557.97   
31267      Removal of nasal sinus tissue using an endoscope             2,454,520.71   
30520      Reshaping of nasal cartilage                                 3,822,391.75   
30140      Removal of nasal air passage                                 4,090,571.97   
15260      Relocation of patient skin to nose, ears, eyeli...           3,816,163.30   
14301      Tissue transfer repair of wound (30.1 to 60.0 s...           3,422,400.08   
14040      Tissue transfer repair of wound (10 sq centimet...           4,175,118.98   
31575      Diagnostic examination of voice box using flexi...           4,095,655.56   
31276      Exploration of nasal sinus using an endoscope                3,551,553.36   
14041      Tissue transfer repair of wound (10.1 to 30.0 s...           2,481,173.43   
60500      Removal or exploration of parathyroid glands                 1,466,292.92   
31231      Diagnostic examination of nasal passages using ...           1,561,172.64   
31579      Examination to assess movement of vocal cord fl...           1,329,457.32   
69436      Incision of eardrum with insertion of eardrum t...           1,291,093.18   
31256      Incision of nasal sinus using an endoscope                   1,021,363.93   
31525      Diagnostic examination of voice box using an en...             548,628.32   
11042      Removal of skin and tissue first 20 sq cm or less              509,299.24   
31237      Biopsy or removal of nasal polyp or tissue usin...             689,656.89   
30465      Widening of nasal passage                                      423,255.25   
15004      Preparation of graft site of face, scalp, eyeli...             492,031.70   
20926      Tissue graft                                                   426,939.03   
21235      Obtaining ear cartilage for grafting                           268,245.90   
31254      Partial removal of nasal sinus using an endoscope              278,635.35   
31288      Removal of nasal sinus tissue using an endoscope               460,441.25   
60240      Removal of thyroid                                             559,710.52   
31571      Injection of vocal cords using an endoscope wit...             337,017.46   
31541      Removal of growth of tongue and/or vocal cord s...             428,916.69   
15120      Skin graft of face, scalp, eyelids, mouth, neck...             249,466.87   
31253      Complete examination of nose and sinuses using ...                   0.00   
31536      Biopsy of voice box using an endoscope with ope...             286,775.12   
30930      Therapeutic fracture of nasal passages                         266,267.33   
31259      Removal of tissue from sphenoid sinus using an ...                   0.00   
11642      Removal of malignant growth (1.1 to 2.0 centime...             289,427.10   
42415      Partial removal of salivary gland growth with r...             196,315.23   
60220      Total removal of thyroid lobe on one side of th...             168,535.09   
69631      Repair of eardrum and ear canal with opening to...             205,385.67   
30130      Removal of nasal air passage                                   227,895.47   
31257      Complete examination of nose and sinuses and re...                   0.00   
31535      Biopsy of voice box using an endoscope                         181,200.03   
31622      Diagnostic examination of lung airways using an...             185,955.57   
15730      Creation of flap graft to mid

# County analysis

## Load data

In [ ]:
# @title Load spatial coordinates of counties
# Below is necessary for plotting chloropleths. 
with urllib.request.urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# @title Load conversion df between FIPS code and county string
fips2county = pd.read_csv("data/fips2county.tsv", sep="\t", comment='#', dtype=str)

In [ ]:
# @title Load our ENT df of all counties, their info, and the Moran's analysis
# The ent CSV file only contains the counties which are analyzable
df_counties_wide_orig = pd.read_csv("data/2022_04_10 ent initial output.csv", dtype={"FIPS": str})

In [ ]:
df_counties_wide_orig.columns

Index(['FIPS', 'Total Number of Services', 'Total Medicare Payment Amount',
       'Total Number of Services: 2019', 'Total Medicare Payment Amount: 2019',
       'Total Number of Services: 2018', 'Total Medicare Payment Amount: 2018',
       'Total Number of Services: 2017', 'Total Medicare Payment Amount: 2017',
       'Total Number of Services: 2016', 'Total Medicare Payment Amount: 2016',
       'Total Number of Services: 2015', 'Total Medicare Payment Amount: 2015',
       'tot_ratio', '% ASC Procedures: 2019', '% ASC Billing: 2019',
       '% ASC Procedures: 2018', '% ASC Billing: 2018',
       '% ASC Procedures: 2017', '% ASC Billing: 2017',
       '% ASC Procedures: 2016', '% ASC Billing: 2016',
       '% ASC Procedures: 2015', '% ASC Billing: 2015', '% ASC Procedures',
       '% ASC Billing', 'Beneficiaries with Part A and Part B', 'Average Age',
       'Percent Male', 'Percent Non-Hispanic White',
       'Percent African American', 'Percent Hispanic',
       'Percent Eligible

In [ ]:
# @title Merge with the fips 2 county standard data set
df_counties_wide = pd.merge(left=df_counties_wide_orig, right=fips2county, how="left", left_on='FIPS', right_on='CountyFIPS')
# Insert a county "County, ST" col (i.e. "Monmouth, NJ" or "Champaign, IL") for ease
df_counties_wide.insert(1, "County_St", df_counties_wide["CountyName"].astype(str) + ", " + df_counties_wide["StateAbbr"].astype(str))

cols_renamed={
    "Average Age": "Average Age (years)",
    'Percent Male': "% Male",
    'Percent Non-Hispanic White': "% Non-Hispanic White",
    'Percent African American': "% African American",
    'Percent Hispanic': "% Hispanic",
    'Percent Eligible for Medicaid': "% Eligible for Medicaid",
    'pct_poverty': "% Poverty",
    'median_house_income': "Median Household Income",
    "Pct_wthout_high_diploma": "% without High School Graduation",
    'unemployment': "Unemployment Rate",
    'pct_uninsured': "% Uninsured",
    'tabacco': "% Tobacco Use",
    'obesity': "% Obesity",
    #"Asthma": "% with Asthma",
    '2013_Rural_urban_cont_code': "RUCA",
    'pop': "Overall Population",
    'Beneficiaries with Part A and Part B': "Medicare Beneficiaries Population",
    'Population Density': "Overall Population Density",
    'Medicare Population Density': "Medicare Population Density",
    "Moran I score for ACS billing fraction":  "Moran I for ASC billing fraction",  # It is "ASC" not "ACS"
}
df_counties_wide = df_counties_wide.rename(columns=cols_renamed)


In [ ]:
info_simple = ["FIPS", "CountyName","StateAbbr", "% ASC Billing"]
info_main = ["FIPS", "County",	"StateFIPS", "Total Medicare Payment Amount", "% ASC Procedures", "% ASC Billing",	"CountyFIPS_3",	"CountyName",	"StateName",	"CountyFIPS",	"StateAbbr",	"STATE_COUNTY", "Moran I for ASC billing fraction"]

df_counties_wide_simple=df_counties_wide[info_simple]
df_counties_wide_main=df_counties_wide[info_main]

# Display with all the columns
with pd.option_context('display.max_rows', 3, 'display.max_columns', None): 
    display(df_counties_wide_simple)


,FIPS,CountyName,StateAbbr,% ASC Billing
0,01017,Chambers,AL,0.000000
...,...,...,...,...
940,51041,Chesterfield,VA,47.027037


## Create long df from wide df- i.e. separate out the year columns into different rows

In [ ]:
col_categories = ["Total Number of Services:", "Total Medicare Payment Amount:", "% ASC Procedures:", "% ASC Billing:"]
cols_to_keep = ["FIPS","County_St"]  # columns to keep in every subgroup so you can line up extra info later

# Create list of df's to combine later, each df is from melting of one category of columns
df_counties_longs = []

# Convert each type of category to long format in separate dataframes
for col_category in col_categories:
        df_counties_long = df_counties_wide.melt(id_vars=cols_to_keep, 
                               var_name="Year", 
                               value_vars=[f"{col_category} {year}" for year in range(2015, 2019 +1)], 
                               value_name=f"{col_category} in Year",
                               )
        df_counties_long["Year"] = df_counties_long["Year"].replace({ f"{col_category} {year}":f"{year}" for year in range(2015, 2019 +1)})
        df_counties_longs.append(df_counties_long)

# Merge the separate category dataframes
df_counties_long = df_counties_longs[0]
for ind in range(1,len(df_counties_longs)):
    df_counties_long = pd.merge(left=df_counties_long, right=df_counties_longs[ind], how="outer", on=(cols_to_keep+["Year"]) )

# Merge with the overall wide dataframe to keep those other values
df_counties_long = pd.merge(left=df_counties_long, 
                   right=df_counties_wide.drop([f"{col_category} {year}" for year in range(2015, 2019 +1) for col_category in col_categories], axis=1), 
                   how="left", on=cols_to_keep)

In [ ]:
df_counties_long

,FIPS,County_St,Year,Total Number of Services: in Year,Total Medicare Payment Amount: in Year,% ASC Procedures: in Year,% ASC Billing: in Year,Total Number of Services,Total Medicare Payment Amount,tot_ratio,...,Medicare Population Density,Moran I score for ACS billing fraction,County,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY
0,01017,"Chambers, AL",2015,0.0,0.00,0.000000,0.000000,408.0,30064.800000,14.196990,...,14.230610,Non Significant,Chambers,01,017,Chambers,Alabama,01017,AL,AL | CHAMBERS
1,01033,"Colbert, AL",2015,108.0,10404.39,0.000000,0.000000,272.0,37080.230000,16.000000,...,22.681014,Non Significant,Colbert,01,033,Colbert,Alabama,01033,AL,AL | COLBERT
2,01045,"Dale, AL",2015,0.0,0.00,0.000000,0.000000,12.0,405.210000,0.999104,...,17.700437,Non Significant,Dale,01,045,Dale,Alabama,01045,AL,AL | DALE
3,01083,"Limestone, AL",2015,0.0,0.00,0.000000,0.000000,55.0,9515.590000,4.000000,...,29.157261,Non Significant,Limestone,01,083,Limestone,Alabama,01083,AL,AL | LIMESTONE
4,05145,"White, AR",2015,1217.0,48412.57,0.000000,0.000000,1269.0,52190.220000,11.995594,...,15.224018,Non Significant,White,05,145,White,Arkansas,05145,AR,AR | WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,21073,"Franklin, KY",2019,0.0,0.00,0.000000,0.000000,114.0,7749.960000,3.910144,...,62.858188,Non Significant,Franklin,21,073,Franklin,Kentucky,21073,KY,KY | FRANKLIN
4701,56021,"Laramie, WY",2019,422.0,79083.86,100.000000,100.000000,1784.0,337949.890001,21.000000,...,6.286729,Non Significant,Laramie,56,021,Laramie,Wyoming,56021,WY,WY | LARAMIE
4702,54041,"Lewis, WV",2019,0.0,0.00,0.000000,0.000000,606.0,26648.230000,4.000000,...,10.524948,Low-Low,Lewis,54,041,Lewis,West Virginia,54041,WV,WV | LEWIS
4703,50027,"Windsor, VT",2019,319.0,12093.61,0.000000,0.000000,1132.0,47825.130000,35.000000,...,14.320922,Low-Low,Windsor,50,027,Windsor,Vermont,50027,VT,VT | WINDSOR


## Set up for summaries and save sums

In [ ]:
# sorted_moran_values = df_counties_wide["Moran I for ASC billing fraction"].unique()
sorted_moran_values = ["High-High","Low-Low","Low-High","High-Low","Non Significant"]  # list out specifically so you can get the order you want
sorted_moran_values_all = sorted_moran_values + ["All"]   #[pd.IndexSlice[:]]  # pd.IndexSlice[:]] represents all

moran_frequencies = df_counties_wide["Moran I for ASC billing fraction"].value_counts()[sorted_moran_values]

In [ ]:
summable_groups = [col for col in df_counties_wide.columns if "total" in col.lower()]
summable_groups = summable_groups + ["Overall Population", "Medicare Beneficiaries Population"]
df_wide_sums = df_counties_wide.groupby("Moran I for ASC billing fraction")[summable_groups].sum()
df_wide_sums = df_wide_sums.assign(Counties=moran_frequencies)
df_wide_sums.loc["All"] = df_wide_sums.sum()

df_wide_sums = df_wide_sums[df_wide_sums.columns[::-1]]  # flip column order left-right to be more logical
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(df_wide_sums)

save_df(df_wide_sums, "df_wide_sums")

,Counties,Medicare Beneficiaries Population,Overall Population,Total Medicare Payment Amount: 2015,Total Number of Services: 2015,Total Medicare Payment Amount: 2016,Total Number of Services: 2016,Total Medicare Payment Amount: 2017,Total Number of Services: 2017,Total Medicare Payment Amount: 2018,Total Number of Services: 2018,Total Medicare Payment Amount: 2019,Total Number of Services: 2019,Total Medicare Payment Amount,Total Number of Services
Moran I for ASC billing fraction,,,,,,,,,,,,,,,
High-High,90,"8,252,843","49,290,013","17,139,184","87,102","18,933,282","106,216","20,196,017","104,668","20,980,333","106,943","21,192,147","116,676","98,440,963","521,605"
High-Low,33,"1,530,091","8,804,718","2,888,148","15,446","3,229,780","15,408","3,008,710","14,722","2,804,513","13,691","2,815,553","14,891","14,746,704","74,158"
Low-High,74,"4,004,628","26,958,805","8,693,092","106,478","7,748,456","90,410","7,584,302","90,625","8,247,523","101,380","8,687,247","103,071","40,960,620","491,964"
Low-Low,139,"3,623,493","19,364,704","5,561,819","95,757","4,296,993","65,244","3,946,056","59,105","4,005,969","61,639","4,649,317","72,170","22,460,154","353,915"
Non Significant,605,"25,346,776","163,845,060","57,438,585","472,764","55,995,168","396,368","58,543,009","428,827","60,913,491","429,358","63,638,158","434,911","296,528,411","2,162,228"
All,941,"42,757,830","268,263,300","91,720,829","777,547","90,203,679","673,646","93,278,093","697,947","96,951,828","713,011","100,982,422","741,719","473,136,851","3,603,870"


In [ ]:
!ls outputs/2022_11_20-RSY-geospatial_ENT_analysis_v03 

df_counties_summary_clean.csv	df_wide_sums.csv
df_counties_summary_clean.xlsx	df_wide_sums.xlsx


## Create summary data by Moran category

In [ ]:
categories = ["Total Number of Services","Total Medicare Payment Amount", "% ASC Procedures", "% ASC Billing" ]

df_counties_with_slope = df_counties_wide.copy()
# Calculate regression and sum and mean from individual year later
for category in categories:
    new_df = df_counties_with_slope[ [category + ": " + str(yr) for yr in years] ].apply(calc_regression,axis=1, result_type="expand", args=(years,) )
    df_counties_with_slope[[category+": "+new_col for new_col in new_df.columns ]]=new_df
# To simplify, drop info for specific years unless it was "Mean" and "Slope" categories we just added
for category in categories:
    df_counties_with_slope = df_counties_with_slope.drop(columns=[col for col in df_counties_with_slope.columns if category in col and "Mean" not in col and "Slope" not in col])


df_counties_summary_dict = {}   # create a dict we will concatenate into a df later
# Options: 	[count, mean, std, min, 25%, 50%, 75%, max] assuming default percentiles argument
cols_to_show = ["10%","mean","90%"]
for possible_Moran_value in sorted_moran_values:
    df_counties_summary_dict[possible_Moran_value] = df_counties_with_slope[df_counties_with_slope["Moran I for ASC billing fraction"]==possible_Moran_value].describe(percentiles=[.1,.25,.5,.75,.9]).loc[cols_to_show]
df_counties_summary_dict["All"] = df_counties_with_slope.describe(percentiles=[.1,.25,.5,.75,.9]).loc[cols_to_show]

df_counties_summary = pd.concat(df_counties_summary_dict.values(), axis=0, keys=df_counties_summary_dict.keys())
for possible_Moran_value in sorted_moran_values:
    df_counties_summary.loc[(possible_Moran_value,cols_to_show[0]), "N"] = moran_frequencies[possible_Moran_value]

# Reorder into the sorted order we set above
df_counties_summary = df_counties_summary.loc[sorted_moran_values_all]



## Create a more presentable format
Select out only the columns you want and rename the columns

In [ ]:
df_counties_long.columns

Index(['FIPS', 'County_St', 'Year', 'Total Number of Services: in Year',
       'Total Medicare Payment Amount: in Year', '% ASC Procedures: in Year',
       '% ASC Billing: in Year', 'Total Number of Services',
       'Total Medicare Payment Amount', 'tot_ratio', '% ASC Procedures',
       '% ASC Billing', 'Beneficiaries with Part A and Part B', 'Average Age',
       'Percent Male', 'Percent Non-Hispanic White',
       'Percent African American', 'Percent Hispanic',
       'Percent Eligible for Medicaid', 'Average HCC Score',
       'Hospital Readmission Rate',
       'Emergency Department Visits per 1000 Beneficiaries',
       'Procedures Per Capita Standardized Costs',
       'Procedure Events Per 1000 Beneficiaries', 'metro', 'pct_poverty',
       'median_house_income', 'pop', '2013_Rural_urban_cont_code',
       'Pct_wthout_high_diploma', 'Pct_wth_high_diploma', 'Pct_wth_some_coll',
       'Pct_wth_coll_degree', 'unemployment', 'pct_uninsured', 'fibro',
       'tabacco', 'obesity'

'% ASC Billing:'

In [ ]:
key_cols={
    'Total Number of Services: Mean': 'Number of Services: Annual' ,
    'Total Number of Services: Slope': 'Number of Services: Yearly Change' ,
    'Total Medicare Payment Amount: Mean' : "Medicare Payment: Annual",
    'Total Medicare Payment Amount: Slope' : "Medicare Payment: Yearly Change",
    '% ASC Billing: Mean': '% ASC Billing',
    '% ASC Billing: Slope': '% ASC Billing: Yearly Change',
    "% ASC Procedures: Mean": "% ASC Procedures",
    "% ASC Procedures: Slope": "% ASC Procedures: Yearly Change",
    "Average Age": "Average Age (years)",
    'Percent Male': "% Male",
    'Percent Non-Hispanic White': "% Non-Hispanic White",
    'Percent African American': "% African American",
    'Percent Hispanic': "% Hispanic",
    'Percent Eligible for Medicaid': "% Eligible for Medicaid",
    'pct_poverty': "% Poverty",
    'median_house_income': "Median Household Income",
    "Pct_wthout_high_diploma": "% without High School Graduation",
    'unemployment': "Unemployment Rate",
    'pct_uninsured': "% Uninsured",
    'tabacco': "% Tobacco Use",
    'obesity': "% Obesity",
    #"Asthma": "% with Asthma",
    '2013_Rural_urban_cont_code': "RUCA",
    'pop': "Overall Population",
    'Beneficiaries with Part A and Part B': "Medicare Beneficiaries Population",
    'Population Density': "Overall Population Density",
    'Medicare Population Density': "Medicare Population Density",
}
df_counties_summary_clean = df_counties_summary[key_cols.keys()]
df_counties_summary_clean = df_counties_summary_clean.rename(columns=key_cols).transpose()

with pd.option_context('display.float_format', '{:,.2f}'.format):
    display( df_counties_summary_clean )

save_df(df_counties_summary_clean, "df_counties_summary_clean")

High-High                          Low-Low  \
                                         10%       mean        90%        10%   
Number of Services: Annual             54.38   1,159.12   1,185.06       8.72   
Number of Services: Yearly Change    -115.49      66.53      54.78    -193.22   
Medicare Payment: Annual           16,018.24 218,757.69 439,851.70     798.98   
Medicare Payment: Yearly Change   -22,898.25  11,281.08  26,467.42 -11,950.69   
% ASC Billing                          37.72      69.28      90.97       0.00   
% ASC Billing: Yearly Change           -4.89       1.44       8.11       0.00   
% ASC Procedures                       17.44      51.27      80.33       0.00   
% ASC Procedures: Yearly Change        -6.74       1.34       9.26       0.00   
Average Age (years)                    71.00      72.41      74.00      68.80   
% Male                                 42.93      44.69      46.80      44.12   
% Non-Hispanic White                   63.48      80.69      92.80      77.03   
% African American                      1.32       8.87      19.75       0.31   
% Hispanic                              0.93       5.83      14.06       0.29   
% Eligible for Medicaid                 9.93      16.61      23.62      15.45   
% Poverty                               6.90      12.54      19.14       8.42   
Median Household Income            47,690.34  64,248.34  90,205.08  42,085.80   
% without High School Graduation        6.90      11.62      15.05       5.50   
Unemployment Rate                       3.60       4.76       5.97       2.88   
% Uninsured                             5.90      10.75      16.56       4.64   
% Tobacco Use                           5.00       7.65      10.04       8.20   
% Obesity                              13.00      19.13      25.12      13.08   
RUCA                                    1.00       1.92       3.00       1.00   
Overall Population                102,551.02 547,666.81 914,269.40  36,255.12   
Medicare Beneficiaries Population  15,823.74  91,698.25 143,264.70   8,175.44   
Overall Population Density            129.82     786.57   1,906.68      48.11   
Medicare Population Density            20.49     134.04     265.73      10.74   

                                                         Low-High             \
                                        mean        90%       10%       mean   
Number of Services: Annual            509.23   1,402.84      3.18   1,329.63   
Number of Services: Yearly Change     -36.53      96.40    -97.49       5.62   
Medicare Payment: Annual           32,316.77  82,467.87    535.71 110,704.38   
Medicare Payment: Yearly Change    -1,522.32   5,963.24 -6,221.68     658.62   
% ASC Billing                           1.12       1.83      0.00       2.74   
% ASC Billing: Yearly Change            0.03       0.00     -1.16      -0.51   
% ASC Procedures                        0.45       0.37      0.00       0.93   
% ASC Procedures: Yearly Change        -0.05       0.00     -0.13      -0.22   
Average Age (years)                    70.37      72.00     69.66      71.49   
% Male                                 45.95      47.51     42.75      45.13   
% Non-Hispanic White                   88.03      96.23     53.49      77.02   
% African American                      4.13      10.02      0.65      12.98   
% Hispanic                              1.22       2.31      0.57       5.69   
% Eligible for Medicaid                23.85      33.66     10.35      19.00   
% Poverty                              13.96      20.09      8.28      14.25   
Median Household Income            54,240.51  68,665.08 41,664.62  59,892.96   
% without High School Graduation        9.98      14.50      7.43      12.56   
Unemployment Rate                       4.58       6.32      3.23       4.67   
% Uninsured                             7.27      11.10      6.70      11.58   
% Tobacco Use                          11.62      16.52      5.66       8.58   
% Obes